# Schedule Interpretation: Critical Evaluation of Schedule Recommendations

This notebook demonstrates how to **interpret and evaluate schedule recommendations** from models.

Understanding schedule interpretation is critical because:
- Schedule recommendations show assignments, but numbers alone don't tell the whole story
- You need to evaluate whether schedules meet objectives and respect constraints
- You need to understand what tradeoffs the model made
- You need to assess whether schedules are robust to uncertainty
- Good interpretation prevents implementing poor schedules


## Key Concepts

**Interpretation Checklist**:
1. **Feasibility**: Are all constraints respected? (time, availability, capabilities, fatigue)
2. **Tradeoffs**: What objectives were prioritized? What was sacrificed?
3. **Robustness**: What happens if conditions change? (demand variability, absences)
4. **Patterns**: Are there fairness issues? Fatigue violations? Over-constraining?

**Common Interpretation Mistakes**:
- Assuming optimal means best in practice
- Ignoring constraints the model might have missed
- Not evaluating tradeoffs
- Assuming schedules are robust to uncertainty
- Not checking for fairness and human factors

**Critical insight**: Schedule interpretation requires critical evaluation, not blind acceptance. The model's recommendation is a starting point, not the final answer.


## Scenario: Nurse Staffing Schedule Review

You receive a nurse staffing schedule recommendation from a scheduling model. The schedule shows nurse assignments across shifts for one week.

**Your task**: Evaluate this schedule using the interpretation checklist to determine if it's acceptable or needs adjustment.

**Constraints to check**:
- Maximum 5 consecutive working days
- Minimum 10 hours between shifts
- Maximum 60 hours per week
- Nurses must have required certifications for assigned units

**Objectives to evaluate**:
- Cost minimization (was this prioritized?)
- Service quality (are staffing levels adequate?)
- Fairness (is work distributed equitably?)


## Step 1: Install Required Packages (Colab)


In [1]:
%pip install matplotlib pandas numpy -q



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


## Step 2: Import Libraries


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Step 3: Load Sample Schedule

Let's create a sample schedule with some issues to identify:


In [3]:
# Sample schedule: 5 nurses, 7 days, 3 shifts per day
# Schedule shows which nurse works which shift
# 1 = working, 0 = off

schedule_data = {
    'Nurse': ['Alice', 'Bob', 'Carol', 'David', 'Eve'],
    'Mon_Day': [1, 1, 0, 0, 1],
    'Mon_Eve': [0, 1, 1, 0, 1],
    'Mon_Night': [0, 0, 1, 1, 0],
    'Tue_Day': [1, 1, 0, 0, 1],
    'Tue_Eve': [0, 1, 1, 0, 1],
    'Tue_Night': [0, 0, 1, 1, 0],
    'Wed_Day': [1, 1, 0, 0, 1],
    'Wed_Eve': [0, 1, 1, 0, 1],
    'Wed_Night': [0, 0, 1, 1, 0],
    'Thu_Day': [1, 1, 0, 0, 1],
    'Thu_Eve': [0, 1, 1, 0, 1],
    'Thu_Night': [0, 0, 1, 1, 0],
    'Fri_Day': [1, 1, 0, 0, 1],
    'Fri_Eve': [0, 1, 1, 0, 1],
    'Fri_Night': [0, 0, 1, 1, 0],
    'Sat_Day': [1, 1, 0, 0, 1],
    'Sat_Eve': [0, 1, 1, 0, 1],
    'Sat_Night': [0, 0, 1, 1, 0],
    'Sun_Day': [1, 1, 0, 0, 1],
    'Sun_Eve': [0, 1, 1, 0, 1],
    'Sun_Night': [0, 0, 1, 1, 0]
}

schedule_df = pd.DataFrame(schedule_data)

print("SAMPLE SCHEDULE:")
print("=" * 80)
print("1 = Working, 0 = Off")
display(schedule_df)

# Calculate hours per nurse
shifts = [col for col in schedule_df.columns if col != 'Nurse']
schedule_df['Total_Shifts'] = schedule_df[shifts].sum(axis=1)
schedule_df['Total_Hours'] = schedule_df['Total_Shifts'] * 8  # 8 hours per shift

print("\nHOURS PER NURSE:")
print("=" * 80)
display(schedule_df[['Nurse', 'Total_Shifts', 'Total_Hours']])


SAMPLE SCHEDULE:
1 = Working, 0 = Off


,Nurse,Mon_Day,Mon_Eve,Mon_Night,Tue_Day,Tue_Eve,Tue_Night,Wed_Day,Wed_Eve,Wed_Night,...,Thu_Night,Fri_Day,Fri_Eve,Fri_Night,Sat_Day,Sat_Eve,Sat_Night,Sun_Day,Sun_Eve,Sun_Night
0,Alice,1,0,0,1,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0
1,Bob,1,1,0,1,1,0,1,1,0,...,0,1,1,0,1,1,0,1,1,0
2,Carol,0,1,1,0,1,1,0,1,1,...,1,0,1,1,0,1,1,0,1,1
3,David,0,0,1,0,0,1,0,0,1,...,1,0,0,1,0,0,1,0,0,1
4,Eve,1,1,0,1,1,0,1,1,0,...,0,1,1,0,1,1,0,1,1,0



HOURS PER NURSE:


,Nurse,Total_Shifts,Total_Hours
0,Alice,7,56
1,Bob,14,112
2,Carol,14,112
3,David,7,56
4,Eve,14,112


## Step 4: Check Feasibility - Constraint Violations

Let's systematically check for constraint violations:


In [4]:
# Check 1: Maximum hours per week (60 hours)
max_hours = 60
violations_hours = schedule_df[schedule_df['Total_Hours'] > max_hours]

print("FEASIBILITY CHECK 1: Maximum Hours per Week")
print("=" * 70)
if len(violations_hours) > 0:
    print(f"✗ VIOLATION: {len(violations_hours)} nurse(s) exceed {max_hours} hours/week")
    display(violations_hours[['Nurse', 'Total_Hours']])
else:
    print(f"✓ All nurses within {max_hours} hours/week limit")

# Check 2: Maximum consecutive days (5 days)
print("\nFEASIBILITY CHECK 2: Maximum Consecutive Days")
print("=" * 70)
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
consecutive_violations = []

for idx, row in schedule_df.iterrows():
    nurse = row['Nurse']
    # Check each day if nurse worked
    worked_days = []
    for day in days:
        day_shifts = [col for col in schedule_df.columns if col.startswith(day)]
        if row[day_shifts].sum() > 0:  # Worked at least one shift
            worked_days.append(day)
    
    # Check for consecutive days > 5
    max_consecutive = 0
    current_consecutive = 0
    for i in range(len(worked_days)):
        if i == 0 or (days.index(worked_days[i]) == days.index(worked_days[i-1]) + 1):
            current_consecutive += 1
        else:
            current_consecutive = 1
        max_consecutive = max(max_consecutive, current_consecutive)
    
    if max_consecutive > 5:
        consecutive_violations.append({'Nurse': nurse, 'Max_Consecutive': max_consecutive})

if len(consecutive_violations) > 0:
    print("✗ VIOLATION: Some nurses work more than 5 consecutive days")
    violations_df = pd.DataFrame(consecutive_violations)
    display(violations_df)
else:
    print("✓ All nurses within 5 consecutive days limit")

# Check 3: Minimum rest between shifts (10 hours)
print("\nFEASIBILITY CHECK 3: Minimum Rest Between Shifts (10 hours)")
print("=" * 70)
rest_violations = []

# Simplified check: if someone works night shift and next day shift, that's < 10 hours
# Night shift ends at 7 AM, day shift starts at 7 AM = 0 hours rest
for idx, row in schedule_df.iterrows():
    nurse = row['Nurse']
    for i, day in enumerate(days[:-1]):  # Check up to second to last day
        next_day = days[i+1]
        if row[f'{day}_Night'] == 1 and row[f'{next_day}_Day'] == 1:
            rest_violations.append({
                'Nurse': nurse,
                'Violation': f'{day} Night → {next_day} Day (0 hours rest)'
            })

if len(rest_violations) > 0:
    print("✗ VIOLATION: Some nurses have insufficient rest between shifts")
    violations_df = pd.DataFrame(rest_violations)
    display(violations_df)
else:
    print("✓ All nurses have adequate rest between shifts")

print("\n" + "=" * 70)
print("FEASIBILITY SUMMARY:")
print("=" * 70)
total_violations = len(violations_hours) + len(consecutive_violations) + len(rest_violations)
if total_violations > 0:
    print(f"✗ Schedule has {total_violations} constraint violation(s) - INFEASIBLE")
    print("  → Schedule cannot be executed as recommended")
    print("  → Must be adjusted before implementation")
else:
    print("✓ Schedule respects all hard constraints - FEASIBLE")


FEASIBILITY CHECK 1: Maximum Hours per Week
✗ VIOLATION: 3 nurse(s) exceed 60 hours/week


,Nurse,Total_Hours
1,Bob,112
2,Carol,112
4,Eve,112



FEASIBILITY CHECK 2: Maximum Consecutive Days
✗ VIOLATION: Some nurses work more than 5 consecutive days


,Nurse,Max_Consecutive
0,Alice,7
1,Bob,7
2,Carol,7
3,David,7
4,Eve,7



FEASIBILITY CHECK 3: Minimum Rest Between Shifts (10 hours)
✓ All nurses have adequate rest between shifts

FEASIBILITY SUMMARY:
✗ Schedule has 8 constraint violation(s) - INFEASIBLE
  → Schedule cannot be executed as recommended
  → Must be adjusted before implementation


In [5]:
# Analyze what the schedule optimized for
hourly_wage = 45
total_cost = schedule_df['Total_Hours'].sum() * hourly_wage
avg_hours = schedule_df['Total_Hours'].mean()
std_hours = schedule_df['Total_Hours'].std()

# Check fairness (coefficient of variation)
fairness_cv = std_hours / avg_hours if avg_hours > 0 else 0

print("TRADEOFF ANALYSIS:")
print("=" * 70)

print(f"\nTotal Cost: ${total_cost:,.2f}")
print(f"Average hours per nurse: {avg_hours:.1f}")
print(f"Hours range: {schedule_df['Total_Hours'].min():.0f} - {schedule_df['Total_Hours'].max():.0f}")
print(f"Fairness (coefficient of variation): {fairness_cv:.2f} (lower = more fair)")

print("\nInterpretation:")
if fairness_cv > 0.2:
    print("  - High variation in hours → Schedule prioritizes COST over FAIRNESS")
    print("  - Some nurses work much more than others")
else:
    print("  - Low variation in hours → Schedule balances COST and FAIRNESS")

if avg_hours < 35:
    print("  - Low average hours → Schedule prioritizes COST minimization")
elif avg_hours > 45:
    print("  - High average hours → Schedule prioritizes SERVICE QUALITY")
else:
    print("  - Moderate average hours → Schedule balances COST and SERVICE")

print("\nKey Insight:")
print("  - Understanding tradeoffs helps you evaluate if the schedule matches your priorities")
print("  - If the model prioritized cost but you need fairness, the schedule may need adjustment")


TRADEOFF ANALYSIS:

Total Cost: $20,160.00
Average hours per nurse: 89.6
Hours range: 56 - 112
Fairness (coefficient of variation): 0.34 (lower = more fair)

Interpretation:
  - High variation in hours → Schedule prioritizes COST over FAIRNESS
  - Some nurses work much more than others
  - High average hours → Schedule prioritizes SERVICE QUALITY

Key Insight:
  - Understanding tradeoffs helps you evaluate if the schedule matches your priorities
  - If the model prioritized cost but you need fairness, the schedule may need adjustment


## Step 6: Evaluate Robustness

What happens if conditions change?


In [6]:
# Scenario 1: What if one nurse calls in sick?
print("ROBUSTNESS ANALYSIS:")
print("=" * 70)

print("\nScenario 1: One nurse calls in sick")
print("-" * 70)
# Check minimum staffing per shift
min_staffing = {}
for shift in ['Day', 'Eve', 'Night']:
    shift_cols = [col for col in schedule_df.columns if col.endswith(f'_{shift}')]
    min_staffing[shift] = schedule_df[shift_cols].sum(axis=1).min()

print("Minimum staffing per shift (if one nurse is absent):")
for shift, min_staff in min_staffing.items():
    required = 2  # Assume minimum 2 nurses required per shift
    status = "✓ Adequate" if min_staff >= required else "✗ Inadequate"
    print(f"  {shift}: {min_staff} nurses {status}")

# Scenario 2: What if demand is 20% higher than expected?
print("\nScenario 2: Demand 20% higher than expected")
print("-" * 70)
demand_increase = 0.2
current_avg_staffing = schedule_df[shifts].sum(axis=1).mean() / len(shifts)
required_staffing = current_avg_staffing * (1 + demand_increase)

print(f"Current average staffing: {current_avg_staffing:.1f} nurses per shift")
print(f"Required with 20% demand increase: {required_staffing:.1f} nurses per shift")
if required_staffing > current_avg_staffing:
    print(f"✗ Schedule is FRAGILE - cannot handle demand increase")
    print(f"  → Would need {required_staffing - current_avg_staffing:.1f} additional nurses")
else:
    print(f"✓ Schedule has buffer capacity for demand variability")

print("\nKey Insight:")
print("  - Robust schedules can handle variability and unexpected events")
print("  - Fragile schedules break down when conditions change")
print("  - Consider adding buffer capacity if robustness is important")


ROBUSTNESS ANALYSIS:

Scenario 1: One nurse calls in sick
----------------------------------------------------------------------
Minimum staffing per shift (if one nurse is absent):
  Day: 0 nurses ✗ Inadequate
  Eve: 0 nurses ✗ Inadequate
  Night: 0 nurses ✗ Inadequate

Scenario 2: Demand 20% higher than expected
----------------------------------------------------------------------
Current average staffing: 0.5 nurses per shift
Required with 20% demand increase: 0.6 nurses per shift
✗ Schedule is FRAGILE - cannot handle demand increase
  → Would need 0.1 additional nurses

Key Insight:
  - Robust schedules can handle variability and unexpected events
  - Fragile schedules break down when conditions change
  - Consider adding buffer capacity if robustness is important


## Step 7: Identify Patterns

Look for patterns that might indicate problems:


In [7]:
# Pattern 1: Unfair distribution
print("PATTERN ANALYSIS:")
print("=" * 70)

print("\nPattern 1: Work Distribution")
print("-" * 70)
hours_range = schedule_df['Total_Hours'].max() - schedule_df['Total_Hours'].min()
if hours_range > 15:
    print(f"✗ UNFAIR: Hours range from {schedule_df['Total_Hours'].min()} to {schedule_df['Total_Hours'].max()} (range: {hours_range})")
    print("  → Some nurses work much more than others")
    print("  → This may create morale problems")
else:
    print(f"✓ FAIR: Hours are relatively evenly distributed (range: {hours_range})")

# Pattern 2: Always same people on undesirable shifts
print("\nPattern 2: Undesirable Shift Assignment")
print("-" * 70)
# Check if same people always work nights/weekends
night_shifts = [col for col in schedule_df.columns if 'Night' in col]
weekend_shifts = [col for col in schedule_df.columns if 'Sat' in col or 'Sun' in col]

schedule_df['Night_Shifts'] = schedule_df[night_shifts].sum(axis=1)
schedule_df['Weekend_Shifts'] = schedule_df[weekend_shifts].sum(axis=1)

night_inequality = schedule_df['Night_Shifts'].std() / schedule_df['Night_Shifts'].mean() if schedule_df['Night_Shifts'].mean() > 0 else 0
weekend_inequality = schedule_df['Weekend_Shifts'].std() / schedule_df['Weekend_Shifts'].mean() if schedule_df['Weekend_Shifts'].mean() > 0 else 0

if night_inequality > 0.5:
    print("✗ UNFAIR: Night shifts are not evenly distributed")
    print("  → Some nurses always work nights, others never do")
    print("  → This creates fairness problems")
else:
    print("✓ FAIR: Night shifts are relatively evenly distributed")

if weekend_inequality > 0.5:
    print("✗ UNFAIR: Weekend shifts are not evenly distributed")
else:
    print("✓ FAIR: Weekend shifts are relatively evenly distributed")

print("\nKey Insight:")
print("  - Patterns reveal fairness and human factor issues")
print("  - Even if mathematically optimal, unfair patterns can cause problems")
print("  - Look for patterns that might lead to resistance or turnover")


PATTERN ANALYSIS:

Pattern 1: Work Distribution
----------------------------------------------------------------------
✗ UNFAIR: Hours range from 56 to 112 (range: 56)
  → Some nurses work much more than others
  → This may create morale problems

Pattern 2: Undesirable Shift Assignment
----------------------------------------------------------------------
✗ UNFAIR: Night shifts are not evenly distributed
  → Some nurses always work nights, others never do
  → This creates fairness problems
✓ FAIR: Weekend shifts are relatively evenly distributed

Key Insight:
  - Patterns reveal fairness and human factor issues
  - Even if mathematically optimal, unfair patterns can cause problems
  - Look for patterns that might lead to resistance or turnover


## Summary: Schedule Interpretation Checklist

**1. Check Feasibility**:
- Are all hard constraints respected? (time, availability, capabilities, fatigue)
- If not, the schedule is infeasible and cannot be executed

**2. Assess Tradeoffs**:
- What objectives did the model prioritize? (cost, service, fairness)
- What was sacrificed to achieve those priorities?
- Do these tradeoffs match your priorities?

**3. Evaluate Robustness**:
- What happens if demand is higher than expected?
- What happens if someone is unavailable?
- Is the schedule fragile or robust?

**4. Identify Patterns**:
- Are there fairness issues? (uneven work distribution)
- Are there fatigue problems? (too many consecutive days)
- Are undesirable shifts always assigned to the same people?

**Practical Implications**:
- Never blindly accept schedule recommendations
- Use the interpretation checklist systematically
- Adjust schedules that don't match your priorities or context
- Communicate tradeoffs to stakeholders
- Remember: optimal ≠ best in practice
